In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from main import *
from models.CLM import CLM
from models.base_model import BaseOrdinalModel
import importlib
import sys
_ = importlib.reload(sys.modules['main'])
_ = importlib.reload(sys.modules['models.CLM'])
_ = importlib.reload(sys.modules['models.base_model'])


In [19]:
X,y =load_data("winequality-red.csv")
X,y

(      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
 0               7.4             0.700         0.00             1.9      0.076   
 1               7.8             0.880         0.00             2.6      0.098   
 2               7.8             0.760         0.04             2.3      0.092   
 3              11.2             0.280         0.56             1.9      0.075   
 4               7.4             0.700         0.00             1.9      0.076   
 ...             ...               ...          ...             ...        ...   
 1594            6.2             0.600         0.08             2.0      0.090   
 1595            5.9             0.550         0.10             2.2      0.062   
 1596            6.3             0.510         0.13             2.3      0.076   
 1597            5.9             0.645         0.12             2.0      0.075   
 1598            6.0             0.310         0.47             3.6      0.067   
 
       free su

In [20]:
from sklearn.utils.estimator_checks import check_estimator, is_regressor
model = load_model("CLM",link_function="logit")
model.fit(X,y)
is_regressor(model)

False

In [ ]:
#create artficial dataset with features age - numerical, height - numerical, gender - categorical (m/w/d) and predict health risk - ordinal (low, medium, high)
np.random.seed(42)
n = 1000
age = np.random.randint(18,100,n)
height = np.random.randint(140,210,n)
gender = np.random.choice(["m","w","d"],n)
#calculate continuous health risk
health_risk =  0.3*np.array([1 if g == "w" else 0 for g in gender]) + 0.3*np.random.randn(n) + 0.6*age/100 + 0.1*height/200
#transform continuous health risk to ordinal health risk
health_risk = pd.qcut(health_risk,3,labels=[0,1,2])
df = pd.DataFrame({"gender":gender, "health_risk":health_risk})
df.to_csv("data/dummy.csv",index=False,sep=";")

In [22]:
from models.CLM import CLM
X,y = load_data("dummy.csv")

model = load_model("CLM",link_function="logit")

model.fit(X,y)

y_pred = model.predict(X)

y_prob = model.predict_proba(X)

y_pred,y_prob




(array([2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1,
        2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 1, 0, 2,
        1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1,
        2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2,
        2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 1, 0, 2, 2, 0, 2, 2, 2, 1,
        2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 1, 0, 2, 1, 2, 2, 2, 2, 2, 0, 1,
        1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 0, 1, 1, 2, 1, 2, 2, 2,
        0, 1, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0,
        2, 0, 0, 2, 2, 2, 0, 0, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1,
        2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2,
        2, 2, 0, 0, 2, 2, 1, 1, 2, 0, 1, 2, 0, 0, 2, 0, 2, 2, 1, 0, 1, 2,
        2, 0, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 1, 1, 2, 1, 0, 0, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 1, 0, 2, 2, 

In [26]:
model.predict(X)

array([2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 1, 0, 2,
       1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1,
       2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2,
       2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2, 1, 0, 2, 2, 0, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 1, 0, 2, 1, 2, 2, 2, 2, 2, 0, 1,
       1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 0, 1, 1, 2, 1, 2, 2, 2,
       0, 1, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0,
       2, 0, 0, 2, 2, 2, 0, 0, 2, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1,
       2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2,
       2, 2, 0, 0, 2, 2, 1, 1, 2, 0, 1, 2, 0, 0, 2, 0, 2, 2, 1, 0, 1, 2,
       2, 0, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 1, 2, 1, 0, 0, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 1, 2,